In [ ]:
import pandas as pd
import ta
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
gaz = pd.read_csv('Data/Dutch TTF Natural Gas Futures - Données Historiques (1).csv')
gaz['Date']=pd.to_datetime(gaz['Date'])
lists = ['Dernier', 'Ouv.', ' Plus Haut', 'Plus Bas', 'Vol.','Variation %']
for i in lists: 
    gaz[i] = gaz[i].str.replace(',', '.')
gaz.columns

In [ ]:
gaz['Dernier'] = pd.to_numeric(gaz['Dernier'], errors='coerce')

gaz['rsi'] = ta.momentum.rsi(gaz['Dernier'], window=14)
gaz['macd'] = ta.trend.macd_diff(gaz['Dernier'])
gaz['ema20'] = ta.trend.ema_indicator(gaz['Dernier'], window=20)
gaz['ema50'] = ta.trend.ema_indicator(gaz['Dernier'], window=50)

gaz

In [ ]:
def convert_k_m_to_numeric(value):
    """
    Convert values with 'K' or 'M' suffix to float numbers.
    Args:
    - value: The string or numeric value to convert.
    
    Returns:
    - The converted value as float if 'K' or 'M' was found; otherwise, the original value.
    """
    if isinstance(value, str):  # Only process strings
        if value.endswith('K'):
            return float(value[:-1]) * 1e3
        elif value.endswith('M'):
            return float(value[:-1]) * 1e6
    return value

In [ ]:
gaz['Vol.'] = gaz['Vol.'].apply(convert_k_m_to_numeric)
for column in gaz.columns:
    gaz[column] = pd.to_numeric(gaz[column], errors='coerce')
assert gaz.applymap(np.isreal).all().all(), "Non-numeric data found in the dataset."

In [ ]:
from src.Features import FeatureEngineer

feature_engineer = FeatureEngineer(gaz)

feature_engineer.calculate_technical_indicators()  # Calculate RSI, MACD, EMA20, EMA50

feature_engineer.normalize_features()  # Normalize the features
features_vector = feature_engineer.construct_feature_vector()  # Get the feature vector

In [ ]:
# make a copy of gaz and fill NaN values with 0
gaz_copy = gaz.copy()
gaz_copy.fillna(0, inplace=True)

gaz_copy.head()

In [ ]:
import numpy as np
import pandas as pd

from src.Environment import TradingEnvironment


# Example usage:

# Define a simple policy function (replace this with your PPO agent's policy for real testing)
def sample_policy(state):
    # This is a dummy policy that randomly decides to buy, hold, or sell
    return np.random.uniform(-1, 1)

# Assuming 'gaz_data' is your preprocessed DataFrame including 'Dernier' prices and any features
env = TradingEnvironment(gaz)
backtest_history = env.run_backtest(sample_policy)


In [ ]:
from src.Agent import PPOAgent
from src.ActorCritic import Actor, Critic



# Define the environment and PPO agent parameters
state_size = 100  # Assume 100 features for the state
action_size = 1  # Buy/sell quantity as a single action
action_bound = 1  # Action limit (e.g., normalized quantity between -1 and 1)

# Instantiate the PPOAgent
agent = PPOAgent(state_size=env.state_space, action_size=env.action_space, action_bound=1)

# Now you can access actor and critic through the agent object
actor_model = Actor(state_size=100, action_size=1, action_bound=1)
critic_model = Critic(state_size=100)

# summary(actor_model, input_size=(1, 100))
# summary(critic_model, input_size=(1, 101))

# critic_model.summary()
print(actor_model)
print(critic_model)

In [ ]:
import wandb
from src.Training import train_agent , run_simulation

wandb.init(project="RL Trading", config={
    "episodes": 1000,
    "learning_rate_actor": 1e-4,
    "learning_rate_critic": 1e-3,
    # Add other relevant configuration parameters here
})
config = wandb.config


env = TradingEnvironment(gaz_copy)
agent = PPOAgent(state_size=env.state_space, action_size=env.action_space, action_bound=1)

# Proceed with training and running the simulation
train_agent(env, agent)
run_simulation(env, agent)